# Chapter 8

### Memory Management
- memory refers to RAM, storage refers to hard drive
- memory is a large array of bytes
    - memory and registers are the only storage that the CPU can access directly
    - instructions and data are fetched from memory
    - memory unit sees a stream of addresses and data
- accessing memory is slow (relative to accessing registers)
    - memory is physically distant from the CPU
    - register access occurs in one clock cycle or less
    - cache sits between main memory and CPU registers 
- protection of memory required to ensure correct operation
    - isolate processes from one another and the OS

### Process Protection in Hardware
- protect OS from user processes and user processes from one another
    - give each process its own address space
    - determine legal addresses a process may access
- the simplest way to do this is to use a pair of registers
    - base and limit registers define the process address space
        - base is the smallest legal physical memory address
        - limit is the length of the address space
            - base + limit = highest legal address for the process
    - every time there is a context switch, the base and limit registers are changed
        - OS must ensure that the new base and limit registers are loaded before the process starts executing
        - OS must ensure that the old base and limit registers are saved when the process is swapped out
    - CPU checks every memory access to ensure it is between base and limit
### Address Binding
- address binding of instructions occurs in one of 3 stages
    - compile time
        - if memory location known a priori, absolute code can be generated
            - must recompile code if starting location changes
    - load time
        - load time is when `exec()` is called
        - must generate **relocatable code** if memory location is not known at compile time
            - compiler generates code with relative addresses
            - linker relocates code and data relative to the beginning of the process's address space
            - code can be placed anywhere in memory
        - process can be moved in memory at any time before load time
    - execution time
        - binding delayed until run time if the process can be moved during its execution from one memory segment to another
            - need hardware support for address maps
                - e.g. base and limit registers
            - must be generated for every memory reference
            - mapping is usually done in hardware, but it can be done by the OS
        - process can be moved in memory at any time, even once it is running
        - most expensive binding
            - must generate **relocatable code**
            - must use hardware to map from virtual to physical address
            - must be done for every memory reference
        - fairly common in modern systems
- <img src="images/sourcetobinary.png">

    - per Kulkarni, compile time goes up to linkage and load time starts at load module

### Logical vs Physical Address Space
- logical address is generated by the CPU
    - also referred to as virtual address
- physical address is address seen by the hardware memory unit
    - also referred to as real address
- at compile time and load time, logical and physical addresses are the same
- at execution time, the addresses differ
### Memory Management Unit (MMU)
- hardware device that maps virtual to physical address
- simple MMU scheme
    - start each process' logical address from 0
    - base memory address for each process is stored in the relocation register
    - max logical address is stored in the limit register
    - relocate register added to every logical address
    - logical: 0 - max = physical: R + 0 - R + max
- <img src="images/mmu.png">

- 

### Static & Dynamic Loading
- static
    - entire program and data must be in memory prior to execution
    - program size is limited to size of physical memory
- dynamic
    - routines are not loaded until they are called
    - better memory space utilization
        - unused routines are never loaded
    - useful when large amounts of code are needed to handle infrequently occurring cases
    - no special support from the OS is required
        - implemented through program design/compiler
- more on this later

### Static & Dynamic Linking
- linker combines multiple external references to object files into a single executable
    - e.g. finding library functions
- static linking
    - occurs prior to runtime
    - library modules considered as any other object file
    - linker/loader combines libraries into every binary image in which they may be referenced
    - multiple copies of the library may exist in memory at the same time
    - larger binary, more memory required (potentially)
- dynamic linking
    - linking postponed until execution time
    - small piece of code, *stub*, used to locate the appropriate memory-resident library routine
    - stub replaces itself with the address of the routine
    - OS checks if the routine is in the process' memory address space
        - if not, OS loads the routine into memory
    - only one copy of the library routine in memory at a time
        - shared across all processes that use it
    - such a library is called a *shared library*
    - smaller binary, less memory required (potentially)
    - required libraries must be present on the system
        - if not, the program will not run because the required functions are not built into the binary
- by default, most compilers use dynamic linking
    - using the `--static` flag will force static linking
        - e.g. `gcc --static -o prog prog.c`
        - 

### Swapping
- 